**Food Delivery Time Prediction**

**1. Data Preprocessing + Feature Engineering**

In [22]:
#Importing necessary libraries
import numpy as np
import pandas as pd
from haversine import haversine
from sklearn.preprocessing import StandardScaler

#Importing the dataset
df = pd.read_csv("Food_Delivery_Time_Prediction.csv")

#Extract coordinates
df[['Cust_Lat', 'Cust_Lon']] = df['Customer_Location'].str.extract(r'\((.*), (.*)\)').astype(float)
df[['Rest_Lat', 'Rest_Lon']] = df['Restaurant_Location'].str.extract(r'\((.*), (.*)\)').astype(float)

#Calculate Distance
df['Distance_km'] = df.apply(lambda row: haversine((row['Cust_Lat'], row['Cust_Lon']), (row['Rest_Lat'], row['Rest_Lon'])), axis=1)

#Time mapping
time_mapping = {
    'Morning': '08:00',
    'Afternoon': '13:00',
    'Evening': '18:00',
    'Night': '22:00'
}

#Time Features
df['Order_Time'] = df['Order_Time'].map(time_mapping)
df['Order_Time'] = pd.to_datetime(df['Order_Time'], format='%H:%M')
df['Hour'] = df['Order_Time'].dt.hour
df['Is_Rush_Hour'] = df['Hour'].apply(lambda x: 1 if 7 <= x <= 9 or 17 <= x <= 20 else 0)

#Encode categorical features
df = pd.get_dummies(df, columns=['Weather_Conditions', 'Traffic_Conditions'], drop_first=True)

#Normalize
scaler = StandardScaler()
df[['Distance_km']] = scaler.fit_transform(df[['Distance_km']])

df.to_csv("processed_food_delivery.csv", index=False)

**2. CNN-Based Classification**

In [32]:
#Importing necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Prepare features
features = ['Cust_Lat', 'Cust_Lon', 'Rest_Lat', 'Rest_Lon', 'Hour', 'Is_Rush_Hour']
x = df[features].values.astype(np.float32)

#Pad to 16
if x.shape[1] < 16:
  x = np.pad(x, ((0, 0), (0,16 - x.shape[1])), mode='constant')

x_images = x.reshape(-1, 4, 4, 1)
df['Delivery_Status'] = df['Delivery_Time'].apply(lambda x: 'Fast' if x <= 30 else 'Delayed')
y = df['Delivery_Status'].apply(lambda x: 1 if x == 'Delayed' else 0).astype(int)
y_cat = to_categorical(y)

#Split
x_train, x_test, y_train, y_test = train_test_split(x_images, y_cat, test_size=0.3, random_state=42)

#CNN Model
model = Sequential([
    Conv2D(16, (2, 2), activation='relu', input_shape=(4,4,1)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_data=(x_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.1251 - loss: 15.3515 - val_accuracy: 0.1000 - val_loss: 7.2751
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2924 - loss: 5.6781 - val_accuracy: 0.7833 - val_loss: 0.4411
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6312 - loss: 1.9559 - val_accuracy: 0.9000 - val_loss: 0.7096
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7896 - loss: 1.3719 - val_accuracy: 0.9000 - val_loss: 0.9308
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8108 - loss: 1.6680 - val_accuracy: 0.9000 - val_loss: 0.8705
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8018 - loss: 1.4051 - val_accuracy: 0.9000 - val_loss: 0.7731
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7788 - loss: 1.7348 - val_accuracy: 0.9000 - val_loss: 0.6255
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8388 - loss: 1.0543 - val_accuracy: 0.9000 - val_loss: 0.5297

**3. Evaluation + Comparison**

In [34]:
#Imporing libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score


# Logistic Regression comparison
x_flat = x_images.reshape(x_images.shape[0], -1)
y_flat = np.argmax(y_cat, axis=1)

lr = LogisticRegression(max_iter=1000)
scores = cross_val_score(lr, x_flat, y_flat, cv=5, scoring='accuracy')
print("Logistic Regression CV Accuracy:", scores.mean())

#CNN perfomance
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred_labels))

Logistic Regression CV Accuracy: 0.8699999999999999
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.90      1.00      0.95        54

    accuracy                           0.90        60
   macro avg       0.45      0.50      0.47        60
weighted avg       0.81      0.90      0.85        60

Confusion Matrix:
 [[ 0  6]
 [ 0 54]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
